# Лабораторная работа №5
## Выполнил студент группы БФИ2302 Филимонов Михаил Алексеевич

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [1]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [ ]:
import time

# Реализация алгоритма Кнута-Морриса-Пратта (КМП) (оптимизации поиска подстроки в строке за счёт предварительной обработки образца и «умных» сдвигов)
# (Сравнение строк слева направо и сдвиг на вычисляемое число позиций)
def kmp_search(text, pattern, case_sensitive=True):
    # Если регистр не важен, приводим обе строки к нижнему регистру
    if not case_sensitive:
        text = text.lower()
        pattern = pattern.lower()

    # Функция для создания префикс-функции (также известной как таблица "частичных совпадений")
    def compute_prefix_function(pattern):
        prefix = [0] * len(pattern)  # Инициализируем префикс-функцию нулями
        j = 0  # j - индекс для отслеживания длины текущего префикса/суффикса
        for i in range(1, len(pattern)): # Начинаем с 1, т.к. prefix[0] всегда 0
            # Пока j > 0 и текущий символ в pattern не совпадает с символом на позиции j
            while j > 0 and pattern[i] != pattern[j]:
                # Перемещаемся к предыдущему меньшему префиксу, используя значение префикс-функции
                j = prefix[j - 1]
            # Если символы совпадают, увеличиваем длину префикса/суффикса
            if pattern[i] == pattern[j]:
                j += 1
            # Записываем длину наибольшего префикса/суффикса для текущей позиции i
            prefix[i] = j
        return prefix

    prefix = compute_prefix_function(pattern) # Вычисляем префикс-функцию для паттерна
    j = 0 # j - индекс для отслеживания текущей позиции в pattern
    occurrences = [] # Список для хранения индексов вхождений

    for i in range(len(text)): # Итерируемся по тексту
        # Пока j > 0 и текущий символ в text не совпадает с символом на позиции j в pattern
        while j > 0 and text[i] != pattern[j]:
            # Перемещаемся к предыдущему меньшему префиксу, используя значение префикс-функции
            j = prefix[j - 1]
        # Если символы совпадают, увеличиваем индекс j (переходим к следующему символу в pattern)
        if text[i] == pattern[j]:
            j += 1
        # Если j равно длине pattern, значит, мы нашли вхождение
        if j == len(pattern):
            # Добавляем индекс начала вхождения в список occurrences
            occurrences.append(i - (j - 1))
            # Перемещаемся к предыдущему меньшему префиксу, чтобы найти другие возможные вхождения
            j = prefix[j - 1]

    return occurrences # Возвращаем список индексов вхождений

# Функция для ввода строки и подстроки с консоли
def input_string_and_substring():
    text = input("Введите строку: ")
    pattern = input("Введите подстроку: ")
    case_sensitive = input("Чувствительность к регистру (y/n): ").lower() == 'y' # Определяем, важен ли регистр
    return text, pattern, case_sensitive

# Основная функция программы
def main():
    text, pattern, case_sensitive = input_string_and_substring() # Получаем входные данные

    # Поиск с использованием алгоритма КМП
    start_time = time.time() # Засекаем время начала работы КМП
    kmp_result = kmp_search(text, pattern, case_sensitive) # Вызываем функцию КМП поиска
    kmp_time = time.time() - start_time # Вычисляем время работы КМП

    # Поиск с использованием стандартной функции Python
    start_time = time.time() # Засекаем время начала работы стандартного поиска
    if case_sensitive:
        # Используем list comprehension и startswith для стандартного поиска с учетом регистра
        standard_result = [i for i in range(len(text)) if text.startswith(pattern, i)]
    else:
        # Используем list comprehension и startswith для стандартного поиска без учета регистра
        standard_result = [i for i in range(len(text)) if text.lower().startswith(pattern.lower(), i)]
    standard_time = time.time() - start_time # Вычисляем время работы стандартного поиска

    # Вывод результатов
    print("\nРезультаты поиска:")
    print(f"Алгоритм КМП: {kmp_result}, время: {kmp_time:.6f} секунд") # Выводим результаты КМП и время
    print(f"Стандартный поиск: {standard_result}, время: {standard_time:.6f} секунд") # Выводим результаты стандартного поиска и время

# Запускаем основную функцию, если скрипт запускается напрямую
if __name__ == "__main__":
    main()


Результаты поиска:
Алгоритм КМП: [], время: 0.000000 секунд
Стандартный поиск: [], время: 0.000000 секунд


#### Упрощенный алгоритм Бойера-Мура

In [ ]:
#(упрощает работу за счёт использования таблицы смещений)
# (Сравнение строк справа налево и сдвиг на вычисляемое число позиций)
def boyer_moore_simple(text, pattern):
    n = len(text)  # Длина текста
    m = len(pattern)  # Длина подстроки
    if m == 0:  # Если подстрока пустая, возвращаем пустой список
        return []

    # таблица сдвигов для неверного символа (bad character rule)
    shift = {}  # Создаем пустой словарь для хранения сдвигов
    for i in range(m - 1):  # Заполняем таблицу сдвигов для всех символов подстроки, кроме последнего
        shift[pattern[i]] = m - i - 1  # Сдвиг равен расстоянию от символа до конца подстроки

    # если символа нет в таблице, сдвигаем на всю длину подстроки
    default_shift = m  # Сдвиг по умолчанию, если символ не найден в таблице

    # поиск подстроки
    i = 0  # Индекс для перебора текста
    occurrences = []  # Список для хранения найденных позиций
    while i <= n - m: # Пока не достигли конца текста (с учетом длины подстроки)
        j = m - 1 # Индекс для перебора подстроки (начинаем с конца)
        while j >= 0 and text[i + j] == pattern[j]: # Пока не дошли до начала подстроки и символы совпадают
            j -= 1 # Переходим к предыдущему символу в подстроке
        if j == -1: # Если дошли до начала подстроки, значит, нашли вхождение
            occurrences.append(i) # Добавляем позицию в список
            i += m # Сдвигаемся на длину подстроки (можно оптимизировать, но это простой вариант)
        else:
            # Символы не совпадают, применяем правило плохого символа
            char = text[i + j] # Получаем символ из текста, который не совпал
            if char in shift:  # Если символ есть в таблице сдвигов
                i += shift[char] # Сдвигаемся на величину, указанную в таблице
            else: # Если символа нет в таблице
                i += default_shift # Сдвигаемся на длину подстроки

    return occurrences # Возвращаем список найденных позиций

text = "ABAAABCDABC"
pattern = "ABC"
result = boyer_moore_simple(text, pattern)
print("Найденные позиции:", result)

Найденные позиции: [4, 8]


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [2]:
# (упорядочить костяшки по возрастанию номеров, перемещая их внутри коробки, желательно сделав как можно меньше перемещений)
from collections import deque

# Функция для проверки, является ли головоломка решаемой
def is_solvable(puzzle):
    inversions = 0 # Начальное количество инверсий
    n = len(puzzle) # Количество элементов в головоломке
    for i in range(n): # Цикл по всем элементам головоломки
        for j in range(i + 1, n): # Цикл по всем элементам после i
            if puzzle[i] != 0 and puzzle[j] != 0 and puzzle[i] > puzzle[j]: # Если нашли инверсию
                inversions += 1 # Увеличиваем счетчик инверсий
    # Для головоломки 4x4:
    # Если пустая клетка находится на чётной строке снизу, то количество инверсий должно быть нечётным.
    # Если пустая клетка находится на нечётной строке снизу, то количество инверсий должно быть чётным.
    empty_row = n // 4 - (puzzle.index(0) // 4) # Определяем номер строки (с конца), где находится пустая клетка
    if empty_row % 2 == 0: # Если номер строки четный
        return inversions % 2 != 0 # Головоломка решаема, если количество инверсий нечетно
    else: # Если номер строки нечетный
        return inversions % 2 == 0 # Головоломка решаема, если количество инверсий четно

# Функция для поиска решения
def solve_puzzle(puzzle):
    if not is_solvable(puzzle):
        return []  # Если головоломка нерешаема, возвращаем пустой список

    target = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] # Целевое состояние головоломки
    if puzzle == target:
        return []  # Если головоломка уже решена, возвращаем пустой список

    # Направления движения: вверх, вниз, влево, вправо
    directions = [(-1, 0, 'up'), (1, 0, 'down'), (0, -1, 'left'), (0, 1, 'right')]

    # Очередь для BFS
    queue = deque()
    queue.append((puzzle, []))  # текущее состояние, последовательность шагов

    # Множество для хранения посещённых состояний
    visited = set()
    visited.add(tuple(puzzle)) # Добавляем начальное состояние в множество посещенных состояний

    while queue: # Пока очередь не пуста
        current, path = queue.popleft() # Извлекаем текущее состояние и путь из очереди
        zero_index = current.index(0) # Находим индекс пустой клетки
        zero_row, zero_col = zero_index // 4, zero_index % 4 # Вычисляем координаты пустой клетки

        for dr, dc, move in directions: # Перебираем все возможные направления движения
            new_row, new_col = zero_row + dr, zero_col + dc # Вычисляем координаты новой позиции пустой клетки
            if 0 <= new_row < 4 and 0 <= new_col < 4: # Проверяем, находится ли новая позиция в пределах головоломки
                new_index = new_row * 4 + new_col # Вычисляем индекс новой позиции
                new_puzzle = current.copy() # Создаем копию текущего состояния головоломки
                # Меняем местами 0 и соседний элемент
                new_puzzle[zero_index], new_puzzle[new_index] = new_puzzle[new_index], new_puzzle[zero_index]
                if tuple(new_puzzle) not in visited: # Если новое состояние еще не посещено
                    if new_puzzle == target: # Если новое состояние является целевым
                        return path + [current[new_index]]  # Возвращаем последовательность шагов
                    visited.add(tuple(new_puzzle)) # Добавляем новое состояние в множество посещенных
                    queue.append((new_puzzle, path + [current[new_index]])) # Добавляем новое состояние и путь в очередь

    return []  # Если решение не найдено

# Ввод данных
puzzle = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 15]  # Пример входных данных

# Решение головоломки
solution = solve_puzzle(puzzle)
if solution:
    print("Решение найдено! Последовательность шагов:", solution)
else:
    print("Решение не найдено.")

Решение найдено! Последовательность шагов: [15]


### Вывод